# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators = 32, random_state=1960)
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


AdaBoostClassifier(n_estimators=32, random_state=1960)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::any> get_classes(){
		std::vector<std::any> lClasses = { 0, 1, 2 };

		return lClasses;
	}

	namespace SubModel_0 {
	
		std::vector<std::any> get_classes(){
			std::vector<std::any> lClasses = { 0, 1, 2 };
	
			return lClasses;
		}
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 1 ,  {1.0, 0.0, 0.0 }} ,
				{ 2 ,  {0.0, 0.5, 0.5 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
			int lNodeIndex = (Feature_2 <= 2.449999988079071) ? ( 1 ) : ( 2 );
		
			return lNodeIndex;
		}
		
	
		std::vector<std::string> get_input_names(){
			std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };
	
			return lFeatures;
		}
	
		std::vector<std::string> get_output_names(){
			std::vector<std::string> lOutputs = { 
				"Score_0", "Score_1", "Score_2",
				"Proba_0", "Proba_1", "Proba_2",
				"LogProb

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_139747920017776.exe /tmp/sklearn2sql_cpp_139747920017776.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,15.73635420988418,-4.25809148069161,-11.47826272919256,0.99995324453654,0.00004552405903,0.00000123140443,-0.00004675655653,-9.99726960184443,-13.60735522609490,0,0.99995324453654
1,15.73635420988418,-4.25809148069161,-11.47826272919256,0.99995324453654,0.00004552405903,0.00000123140443,-0.00004675655653,-9.99726960184443,-13.60735522609490,0,0.99995324453654
2,15.73635420988418,-4.25809148069161,-11.47826272919256,0.99995324453654,0.00004552405903,0.00000123140443,-0.00004675655653,-9.99726960184443,-13.60735522609490,0,0.99995324453654
3,15.73635420988418,-4.25809148069161,-11.47826272919256,0.99995324453654,0.00004552405903,0.00000123140443,-0.00004675655653,-9.99726960184443,-13.60735522609490,0,0.99995324453654
4,15.73635420988418,-4

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,-40.048730,16.323013,23.725717,1.000000e-14,0.024095,0.975905,-31.911613,-3.725742,-0.024390,2,0.975905
74,74,-26.783136,17.001654,9.781482,3.024600e-10,0.973663,0.026337,-21.919085,-0.026690,-3.636776,1,0.973663
9,9,15.736354,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
88,88,-26.783136,17.001654,9.781482,3.024600e-10,0.973663,0.026337,-21.919085,-0.026690,-3.636776,1,0.973663
25,25,15.736354,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
5,5,15.736354,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
48,48,15.736354,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
117,117,-40.048730,16.323013,23.725717,1.000000e-14,0.024095,0.975905,-31.911613,-3.725742,-0.024390,2,0.975905
83,83,-33.377175,16.666573,16.710602,6.720000e-12,0.494497,0.505503,-25.726089,-0.704215,-0.682201,2,0.505503
105,105,-40.048730,16.323013,23.725717,1.000000e-14,0.024095,0.975905,-31.911613,-3.725742,-0.024390,2,0.975905


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,3.071868e-16,0.024095,9.759048e-01,-35.719076,-3.725742,-0.024390,2
74,74,NaN,NaN,NaN,4.506947e-11,0.973663,2.633712e-02,-23.822816,-0.026690,-3.636776,1
9,9,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,0
88,88,NaN,NaN,NaN,4.506947e-11,0.973663,2.633712e-02,-23.822816,-0.026690,-3.636776,1
25,25,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,0
5,5,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,0
48,48,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,0
117,117,NaN,NaN,NaN,3.071868e-16,0.024095,9.759048e-01,-35.719076,-3.725742,-0.024390,2
83,83,NaN,NaN,NaN,3.864841e-13,0.494497,5.055033e-01,-28.581686,-0.704215,-0.682201,2
105,105,NaN,NaN,NaN,3.071868e-16,0.024095,9.759048e-01,-35.719076,-3.725742,-0.024390,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,3.071868e-16,0.024095,9.759048e-01,-35.719076,-3.725742,-0.024390,...,16.323013,23.725717,1.000000e-14,0.024095,0.975905,-31.911613,-3.725742,-0.024390,2,0.975905
74,74,NaN,NaN,NaN,4.506947e-11,0.973663,2.633712e-02,-23.822816,-0.026690,-3.636776,...,17.001654,9.781482,3.024600e-10,0.973663,0.026337,-21.919085,-0.026690,-3.636776,1,0.973663
9,9,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,...,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
88,88,NaN,NaN,NaN,4.506947e-11,0.973663,2.633712e-02,-23.822816,-0.026690,-3.636776,...,17.001654,9.781482,3.024600e-10,0.973663,0.026337,-21.919085,-0.026690,-3.636776,1,0.973663
25,25,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,...,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
5,5,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,...,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
48,48,NaN,NaN,NaN,9.999930e-01,0.000007,1.835009e-07,-0.000007,-11.900961,-15.511046,...,-4.258091,-11.478263,9.999532e-01,0.000046,0.000001,-0.000047,-9.997270,-13.607355,0,0.999953
117,117,NaN,NaN,NaN,3.071868e-16,0.024095,9.759048e-01,-35.719076,-3.725742,-0.024390,...,16.323013,23.725717,1.000000e-14,0.024095,0.975905,-31.911613,-3.725742,-0.024390,2,0.975905
83,83,NaN,NaN,NaN,3.864841e-13,0.494497,5.055033e-01,-28.581686,-0.704215,-0.682201,...,16.666573,16.710602,6.720000e-12,0.494497,0.505503,-25.726089,-0.704215,-0.682201,2,0.505503
105,105,NaN,NaN,NaN,3.071868e-16,0.024095,9.759048e-01,-35.719076,-3.725742,-0.024390,...,16.323013,23.725717,1.000000e-14,0.024095,0.975905,-31.911613,-3.725742,-0.024390,2,0.975905


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
